In [1]:
import os
from transformers import AutoTokenizer, AutoModel
import torch

# 打印目录内容，确保 pytorch_model.bin 在其中
model_path = "./zhihan1996/DNABERT-2-117M"
print(os.listdir(model_path))

# 尝试加载模型
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)


/root/miniconda3/envs/dna/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['bert_padding.py', 'config.json', 'LICENSE', 'bert_layers.py', '.gitattributes', 'configuration_bert.py', 'flash_attn_triton.py', 'README.md', 'generation_config.json', 'tokenizer_config.json', 'tokenizer.json', 'pytorch_model.bin']


/root/.cache/huggingface/modules/transformers_modules/DNABERT-2-117M/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at ./zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifi

In [2]:
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
print(inputs)
hidden_states = model(inputs)[0] # [1, sequence_length, 768]

# embedding with mean pooling
embedding_mean = torch.mean(hidden_states[0], dim=0)
print(embedding_mean.shape) # expect to be 768

# embedding with max pooling
embedding_max = torch.max(hidden_states[0], dim=0)[0]
print(embedding_max.shape) # expect to be 768


tensor([[   1,    5,  194,   32,  757, 1239, 2092,  294,   24,  359,   88,   93,
           32,   75,   77,   19,    2]])
torch.Size([768])
torch.Size([768])


# 修改

In [4]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


# 打印目录内容，确保 pytorch_model.bin 在其中
model_path = "./zhihan1996/DNABERT-2-117M"
print(os.listdir(model_path))

# 尝试加载模型
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

# 使用 to() 方法将模型和输入移动到 GPU（如果可用的话）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
inputs = tokenizer(dna, return_tensors='pt').to(device)

print(inputs)
hidden_states = model(**inputs)[0]  # 使用 last_hidden_state 获取输出

# embedding with mean pooling
embedding_mean = torch.mean(hidden_states, dim=1)
print(embedding_mean.shape)  # 期望为 [1, 768]

# embedding with max pooling
embedding_max = torch.max(hidden_states, dim=1).values
print(embedding_max.shape)  # 期望为 [1, 768]


['bert_padding.py', 'config.json', 'LICENSE', 'bert_layers.py', '.gitattributes', 'configuration_bert.py', 'flash_attn_triton.py', 'README.md', 'generation_config.json', 'tokenizer_config.json', 'tokenizer.json', 'pytorch_model.bin']


Some weights of the model checkpoint at ./zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should 

{'input_ids': tensor([[   1,    5,  194,   32,  757, 1239, 2092,  294,   24,  359,   88,   93,
           32,   75,   77,   19,    2]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
torch.Size([1, 768])
torch.Size([1, 768])
